<a href="https://colab.research.google.com/github/monsterhunters/Flux-Colab-GUI/blob/main/Flux_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install Module


import os
os.chdir('/content')
os.system('git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI')
ROOT = "/content/TotoroUI"
os.chdir(ROOT)
!pip install -q torchsde einops diffusers accelerate gradio python-multipart==0.0.12
os.system('pip install torchaudio==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121')
os.system('pip install -U xformers torch --index-url https://download.pytorch.org/whl/cu121')
!apt -y install -qq aria2
os.makedirs('/content/TotoroUI/models/checkpoints', exist_ok=True)
os.makedirs('/content/TotoroUI/models/loras', exist_ok=True)
os.makedirs('/content/TotoroUI/output', exist_ok=True)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8-all-in-one.safetensors -d /content/TotoroUI/models/checkpoints -o flux1-dev-fp8-all-in-one.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux_realism_lora.safetensors -d /content/TotoroUI/models/loras -o flux_realism_lora.safetensors


In [ ]:
# @title RUN
import random, os

ROOT = "/content/TotoroUI"
os.chdir(ROOT)

import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_flux
from totoro import model_management
from urllib.parse import urlparse
import gradio as gr
import tempfile

CheckpointLoaderSimple = NODE_CLASS_MAPPINGS["CheckpointLoaderSimple"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

CHECKPOINT_FOLDER = "/content/TotoroUI/models/checkpoints"
LORA_FOLDER = "/content/TotoroUI/models/loras"

def refresh_options():
    # Dynamically list files in the directories
    checkpoint_options = [f for f in os.listdir(CHECKPOINT_FOLDER) if f.endswith(('.ckpt', '.safetensors'))]
    lora_options = [f for f in os.listdir(LORA_FOLDER) if f.endswith('.safetensors')]
    return gr.update(choices=checkpoint_options), gr.update(choices=lora_options)

def button_action(download_link, civitai_api, selection):
    output_path = set_output_path(selection)
    parsed_url = urlparse(download_link)
    path_components = parsed_url.path.split("/")
    modelVersionId = path_components[-1]
    dlink = f"https://civitai.com/api/download/models/{modelVersionId}?token={civitai_api}"
    os.system(f"aria2c -x 16 -s 16 -k 1M -d {output_path} {dlink}")
    return "File downloaded successfully!"


def set_output_path(selection):
    if selection == "Checkpoint":
        return "/content/TotoroUI/models/checkpoints"
    else:
        return "/content/TotoroUI/models/loras"

def get_files(folder, extensions):
    if not os.path.exists(folder):
        return []
    return [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.endswith(extensions)]

checkpoint_options = get_files(CHECKPOINT_FOLDER, (".ckpt", ".safetensors"))
lora_options = get_files(LORA_FOLDER, (".safetensors",))

# Fallback for empty dropdowns
if not checkpoint_options:
    checkpoint_options = ["No checkpoint files found"]
if not lora_options:
    lora_options = ["No LoRA files found"]

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

def generate(checkpoints_name, lora_name, positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
    try:
        with torch.no_grad(): # Use no_grad for inference
            # Load checkpoint dynamically
            unet, clip, vae = CheckpointLoaderSimple.load_checkpoint(checkpoints_name)

            if seed == 0:
                seed = random.randint(0, 18446744073709551615)
            print(f"Seed: {seed}")

            # Load LoRA model
            unet_lora, clip_lora = LoraLoader.load_lora(unet, clip, lora_name, lora_strength_model, lora_strength_clip)
            cond, pooled = clip_lora.encode_from_tokens(clip_lora.tokenize(positive_prompt), return_pooled=True)
            cond = [[cond, {"pooled_output": pooled}]]
            cond = FluxGuidance.append(cond, guidance)[0]
            noise = RandomNoise.get_noise(seed)[0]
            guider = BasicGuider.get_guider(unet_lora, cond)[0]
            sampler = KSamplerSelect.get_sampler(sampler_name)[0]
            sigmas = BasicScheduler.get_sigmas(unet_lora, scheduler, steps, 1.0)[0]
            latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
            sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
            decoded = VAEDecode.decode(vae, sample)[0].detach()

            # Save image to a temporary file
            image = Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0])
            image.save(f"//content/TotoroUI/output/flux_{seed}.png")

            return f"/content/TotoroUI/output/flux_{seed}.png"


    except Exception as e:
        return None, f"Error during generation: {str(e)}"

with gr.Blocks(analytics_enabled=False) as demo:
    with gr.Row():
        with gr.Column():
            civitai_api = gr.Textbox(label="Civitai API Key")
            download_link = gr.Textbox(label="Input URL")
            radio_group = gr.Radio(
                choices=["Checkpoint", "Lora"],
                label="Output folder",
                value="Checkpoint"
            )
            fn=set_output_path, # Function to handle the selection
            inputs=radio_group, # Input is the radio group
            ordinary_button = gr.Button("Download File")

            positive_prompt = gr.Textbox(
                lines=3,
                interactive=True,
                value="cute anime girl with massive fluffy fennec ears and a big fluffy tail blonde messy long hair blue eyes wearing a maid outfit with a long black dress with a gold leaf pattern and a white apron eating a slice of an apple pie in the kitchen of an old dark victorian mansion with a bright window and very expensive stuff everywhere",
                label="Prompt"
            )
            width = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="Width")
            height = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="Height")
            seed = gr.Slider(minimum=0, maximum=18446744073709551615, value=0, step=1, label="Seed (0 = Random)")
            steps = gr.Slider(minimum=4, maximum=50, value=20, step=1, label="Steps")
            guidance = gr.Slider(minimum=0, maximum=20, value=3.5, step=0.5, label="Guidance")
            lora_strength_model = gr.Slider(minimum=0, maximum=2, value=1.0, step=0.1, label="LoRA Strength (Model)")
            lora_strength_clip = gr.Slider(minimum=0, maximum=2, value=1.0, step=0.1, label="LoRA Strength (Clip)")
            sampler_name = gr.Dropdown(
                ["euler", "heun", "heunpp2", "dpm_2", "lms", "dpmpp_2m", "ipndm", "deis", "ddim", "uni_pc", "uni_pc_bh2"],
                label="Sampler Name",
                value="euler"
            )
            scheduler = gr.Dropdown(
                ["normal", "sgm_uniform", "simple", "ddim_uniform"],
                label="Scheduler",
                value="simple"
            )


        with gr.Column():
            refresh_button = gr.Button("Refresh Checkpoint/Lora")
            checkpoint_dropdown = gr.Dropdown(label="Checkpoint File", choices=checkpoint_options)
            lora_dropdown = gr.Dropdown(label="LoRA File", choices=lora_options)
            generate_button = gr.Button("Generate")
            output_image = gr.Image(label="Generated Image", interactive=False)
            status_message = gr.Textbox(label="Status", interactive=False)

    # Link buttons to actions
    ordinary_button.click(
        fn=button_action,
        inputs=[download_link, civitai_api, radio_group],
        outputs=status_message
    )

    refresh_button.click(
        fn=refresh_options,
        outputs=[checkpoint_dropdown, lora_dropdown]
    )

    generate_button.click(
        fn=generate,
        inputs=[
            checkpoint_dropdown,
            lora_dropdown,
            positive_prompt,
            width,
            height,
            seed,
            steps,
            sampler_name,
            scheduler,
            guidance,
            lora_strength_model,
            lora_strength_clip
        ],
        outputs=output_image
    )

demo.queue().launch(inline=False, share=True, debug=True, allowed_paths=["/content/TotoroUI/output")
demo.queue().launch(allowed_paths=["/content/TotoroUI/output"])

In [ ]:
# @title Clear VRAM
import torch
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_accumulated_memory_stats()

import gc
gc.collect()
